In [3]:
import pandas as pd
import requests
import time
import re
from deep_translator import GoogleTranslator
import re
import time

In [ ]:
#viewingActivity = pd.read_csv("ViewingActivity.csv")
myActivity = pd.read_csv("ViewingActivity.csv")

In [4]:
myActivity.head()

,Duration,Start Time,Profile Name,Country,Bookmark,Latest Bookmark,Supplemental Video Type,Attributes,Device Type,Title
0,00:02:43,2025-10-01 07:20:42,Fiona,US (United States),00:34:46,00:34:46,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
1,00:16:31,2025-10-01 06:29:48,Fiona,US (United States),00:32:03,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
2,00:08:23,2025-10-01 06:06:49,Fiona,US (United States),00:15:32,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
3,00:06:04,2025-10-01 00:48:53,Fiona,US (United States),00:07:09,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
4,00:31:34,2025-09-30 23:15:56,Fiona,US (United States),00:40:02,00:40:02,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 2 (Episode 2)


In [5]:
myActivity = myActivity[
    pd.isna(myActivity['Supplemental Video Type']) &
    pd.notna(myActivity['Profile Name'])
]
myActivity.head()

,Duration,Start Time,Profile Name,Country,Bookmark,Latest Bookmark,Supplemental Video Type,Attributes,Device Type,Title
0,00:02:43,2025-10-01 07:20:42,Fiona,US (United States),00:34:46,00:34:46,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
1,00:16:31,2025-10-01 06:29:48,Fiona,US (United States),00:32:03,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
2,00:08:23,2025-10-01 06:06:49,Fiona,US (United States),00:15:32,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
3,00:06:04,2025-10-01 00:48:53,Fiona,US (United States),00:07:09,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3)
4,00:31:34,2025-09-30 23:15:56,Fiona,US (United States),00:40:02,00:40:02,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 2 (Episode 2)


In [6]:
myActivity.rename(columns={'Title': 'FullTitle'}, inplace=True)
for index, row in myActivity.iterrows():
    if '：' in row['FullTitle']:
        myActivity.at[index, 'FullTitle'] = row['FullTitle'].replace('：', ':')


myActivity['Title'] = myActivity['FullTitle'].str.split(':').str[0]
myActivity['Season'] = myActivity['FullTitle'].str.split(':').str[1]
myActivity['Episode'] = myActivity['FullTitle'].str.split(':').str[2]

In [ ]:
myActivity['Title'].unique()

array(['Mobius', '淚之女王', '浪漫的體質', 'Burning Betrayal', 'Fatal Seduction',
       'Until You Burn', '直到黎明', 'The Naked Director', 'Toy Boy',
       'Bon Appétit, Your Majesty', 'Playing with Fire',
       'My Life With the Walter Boys', '海岸村恰恰恰', '迷離懸案', '陌生號碼',
       'Unknown Number', 'Unspeakable Sins', '你的味道', '菜鳥警察大叔',
       '19/20 成年初體驗', 'Hunter X Hunter (2011)', '暴君的廚師',
       'Our Beloved Summer', 'Melo Movie', 'Wednesday', '絕命殺陣 2', '體能之巔',
       '離線找真愛', 'Our Generation', '駐韓超富族', '母胎單身戀愛大作戰', '魔鬼的計謀',
       'Unknown Title', '玫瑰的故事', '愛在牛津的一年', '高爾夫球也瘋狂 2', '高爾夫球也瘋狂',
       'Dark Desire', 'Diary of a Gigolo', 'Bitch and Rich', '不如海邊吹吹風',
       '樂來越愛你', '名偵探柯南', '童話故事下集', '他們的春夏秋冬', '比天堂還美麗', '蠻好的人生',
       'The Princess’s Gambit', '富都青年', '浴血黑幫', '光逝去的夏天', 'Kpop 獵魔女團',
       'The Prisoner of Beauty', 'Forget You Not', '魷魚遊戲',
       'Kill Me Love Me', '倫敦恐攻', 'Brocade Odyssey', '玩命關頭8',
       'You Are My Secret', 'Feud', '半糖關係 2', 'Stranger Things',
       'The Lady I

In [ ]:
# Clean up the Season column first
def clean_season(value):
    if pd.isna(value):
        return None
    value = str(value).strip()
    match = re.search(r'\d+', value)  # extract the first number
    return match.group(0) if match else None

myActivity['Season'] = myActivity['Season'].apply(clean_season)

In [27]:
fionaActivity = myActivity[myActivity['Profile Name'] == 'Fiona']

In [28]:
TMDB_API_KEY = "db49686bc0b2f60662893560049047f4"

# TMDb endpoints
TMDB_SEARCH_TV_URL = "https://api.themoviedb.org/3/search/tv"
TMDB_SEARCH_MOVIE_URL = "https://api.themoviedb.org/3/search/movie"
TMDB_TV_URL = "https://api.themoviedb.org/3/tv/"
TMDB_MOVIE_URL = "https://api.themoviedb.org/3/movie/"

# === Add new columns if not exist ===
new_columns = [
    'Genre', 'Director', 'Casting', 'Year_of_Release',
    'Country_of_Origin', 'Awards', 'Length', 'Description'
]
for col in new_columns:
    if col not in fionaActivity.columns:
        myActivity[col] = ''

# === Main TMDb enrichment loop ===
for i, row in myActivity.iterrows():
    title = str(row['Title']).strip()
    clean_title = re.sub(r'\b[Ss]eason\s*\d+\b', '', title).strip()

    try:
        # --- 1️⃣ Try searching as a TV show first ---
        params = {'api_key': TMDB_API_KEY, 'query': clean_title, 'language': 'zh-TW'}
        tv_search = requests.get(TMDB_SEARCH_TV_URL, params=params).json()
        results = tv_search.get('results', [])

        if results:
            media_type = 'tv'
            tmdb_id = results[0]['id']
            details_url = f"{TMDB_TV_URL}{tmdb_id}"
        else:
            # --- 2️⃣ If no TV result, try movie search ---
            movie_search = requests.get(TMDB_SEARCH_MOVIE_URL, params=params).json()
            results = movie_search.get('results', [])

            if not results:
                print(f"🚫 Not found: {clean_title}")
                continue

            media_type = 'movie'
            tmdb_id = results[0]['id']
            details_url = f"{TMDB_MOVIE_URL}{tmdb_id}"

        # --- Fetch detailed info ---
        details = requests.get(details_url, params={'api_key': TMDB_API_KEY, 'language':'en-US'}).json()

        # --- Fetch credits (cast + crew) ---
        credits_url = f"{details_url}/credits"
        credits = requests.get(credits_url, params={'api_key': TMDB_API_KEY}).json()

        cast_list = [c['name'] for c in credits.get('cast', [])[:5]]  # top 5 cast
        director_list = [c['name'] for c in credits.get('crew', []) if c.get('job') == 'Director']

        # --- Store extracted info ---
        fionaActivity.at[i, 'Genre'] = ", ".join([g['name'] for g in details.get('genres', [])])
        fionaActivity.at[i, 'Director'] = ", ".join(director_list)
        fionaActivity.at[i, 'Casting'] = ", ".join(cast_list)
        fionaActivity.at[i, 'Year_of_Release'] = details.get('release_date', details.get('first_air_date', ''))[:4]
        fionaActivity.at[i, 'Country_of_Origin'] = ", ".join(details.get('origin_country', []))

        runtime = details.get('runtime', details.get('episode_run_time', []))
        if isinstance(runtime, list):
            runtime = runtime[0] if runtime else ''
        fionaActivity.at[i, 'Length'] = f"{runtime} min" if runtime else ''

        fionaActivity.at[i, 'Description'] = details.get('overview', '')
        fionaActivity.at[i, 'Awards'] = ''  # TMDb doesn't provide awards

        print(f"✅ TMDb found: {clean_title} ({media_type})")

    except Exception as e:
        print(f"❌ Error on {clean_title}: {e}")

    time.sleep(0.5)  # avoid API rate limits

# === Save results ===
fionaActivity.to_csv("netflix_tmdb_only.csv", index=False)
print("🎉 Done! Saved as 'netflix_tmdb_only.csv'")

✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)


C:\Users\fiona\AppData\Local\Temp\ipykernel_25588\2207479121.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fionaActivity.at[i, 'Genre'] = ", ".join([g['name'] for g in details.get('genres', [])])


✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: Burning Betrayal (movie)
✅ TMDb found: Fatal Seduction (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: Until You Burn (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: Mobius (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: Until You Burn (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 浪漫的體質 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 淚之女王 (tv)
✅ TMDb found: 直到黎明 (tv)
✅ TMDb found: Fat

**Data Cleaning after Web Scrapping**

After web scrapping, we have a file named "netflix_tmdb_only.csv", and we can start clean this data for visualization and analysis.

In [ ]:
# delete all therows with null FullTitle
viewings = pd.read_csv("netflix_tmdb_only.csv")
viewings = viewings[viewings['FullTitle'].isna()==False]

In [ ]:
# convert the year column to integer type
viewings['Year_of_Release'] = pd.to_numeric(viewings['Year_of_Release'], errors='coerce').fillna(0).astype(int)
viewings.head(5)

,Duration,Start Time,Profile Name,Country,Bookmark,Latest Bookmark,Supplemental Video Type,Attributes,Device Type,FullTitle,...,Episode,Genre,Director,Casting,Year_of_Release,Country_of_Origin,Awards,Length,Description,english_title
0,00:02:43,2025-10-01 07:20:42,Fiona,US (United States),00:34:46,00:34:46,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3),...,Episode 3 (Episode 3),"Drama, Mystery, Crime",Liu Zhangmu,"Bai Jingting, Janice Man, Song Yang, Zhang Boj...",2025,CN,NaN,45 min,"In Hua'ao City, a brilliant detective with a f...",Mobius
1,00:16:31,2025-10-01 06:29:48,Fiona,US (United States),00:32:03,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3),...,Episode 3 (Episode 3),"Drama, Mystery, Crime",Liu Zhangmu,"Bai Jingting, Janice Man, Song Yang, Zhang Boj...",2025,CN,NaN,45 min,"In Hua'ao City, a brilliant detective with a f...",Mobius
2,00:08:23,2025-10-01 06:06:49,Fiona,US (United States),00:15:32,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3),...,Episode 3 (Episode 3),"Drama, Mystery, Crime",Liu Zhangmu,"Bai Jingting, Janice Man, Song Yang, Zhang Boj...",2025,CN,NaN,45 min,"In Hua'ao City, a brilliant detective with a f...",Mobius
3,00:06:04,2025-10-01 00:48:53,Fiona,US (United States),00:07:09,Not latest view,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 3 (Episode 3),...,Episode 3 (Episode 3),"Drama, Mystery, Crime",Liu Zhangmu,"Bai Jingting, Janice Man, Song Yang, Zhang Boj...",2025,CN,NaN,45 min,"In Hua'ao City, a brilliant detective with a f...",Mobius
4,00:31:34,2025-09-30 23:15:56,Fiona,US (United States),00:40:02,00:40:02,NaN,NaN,Apple iPad Pro 12.9 inch 3rd Gen Wi-Fi iPad,Mobius: Season 1: Episode 2 (Episode 2),...,Episode 2 (Episode 2),"Drama, Mystery, Crime",Liu Zhangmu,"Bai Jingting, Janice Man, Song Yang, Zhang Boj...",2025,CN,NaN,45 min,"In Hua'ao City, a brilliant detective with a f...",Mobius


**Handle More Null Values**

In [13]:
viewings.shape

(2355, 22)

In [16]:
# look at the total null values in each column
viewings.isna().sum()

Duration                      0
Start Time                    0
Profile Name                  0
Country                       0
Bookmark                      0
Latest Bookmark               0
Supplemental Video Type    2355
Attributes                 2285
Device Type                   0
FullTitle                     0
Title                         0
Season                     1086
Episode                      49
Genre                       122
Director                   1671
Casting                     179
Year_of_Release               0
Country_of_Origin             4
Awards                     2355
Length                      829
Description                  92
english_title              1000
dtype: int64

We only want to use the columns that have enough data. We would keep these columns in our datafame: Duration, Start Time, Bookmark, Latest Bookmark, Title, Season, Episode, Genre, Director, Casting, Year_of_Release, Country_of_Origin, Length, Description.

**Data Visualization**

In [10]:
viewings['Genre'].value_counts().head(10)

Genre
Drama                                              581
Drama, Comedy                                      195
Comedy, Drama                                      180
Reality                                            172
Animation, Comedy, Drama                           139
Animation, Action & Adventure, Sci-Fi & Fantasy    110
Action & Adventure, Sci-Fi & Fantasy                66
Comedy                                              55
Sci-Fi & Fantasy, Action & Adventure                53
Animation, Sci-Fi & Fantasy, Action & Adventure     44
Name: count, dtype: int64

In [11]:
viewings['Country_of_Origin'].value_counts().head(10)

Country_of_Origin
KR    765
JP    548
US    405
CN    280
TW    127
ID     66
GB     54
DE     37
TH     32
AU     12
Name: count, dtype: int64